# Nuevo Proyecto Machine Learning 

- Este proyecto será un recomendador donde la persona deberá de poner ciertos features para recomendar un restaurante dentro de los que están en la guía Michelin. 





In [1]:
#Para la manipulación de datos
import pandas as pd
import time
import numpy as np

#Servicio y driver de Chrome de Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#Las opciones que vamos a tener para buscar elementos
from selenium.webdriver.common.by import By

#Para cuando queramos mandar pulsaciones de teclado
from selenium.webdriver.common.keys import Keys

# Para generar datos artificiales 
import random 

# Para crear el recomendador 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

# Para el análsis de la muestra 
import matplotlib.pyplot as plt 
import seaborn as sns 

# Para guardar el modelo 
import pickle

## Extracción de datos

In [4]:
# Creamos la conexión con la página web
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
time.sleep(2)

In [5]:
# Ponemos la página web 
url = 'https://guide.michelin.com/es/es'
driver.get(url)


In [5]:
# Encontrar las cookies
elements_by_ID = driver.find_element(By.ID, 'didomi-notice-agree-button')
print(elements_by_ID.text)


Aceptar y cerrar


In [6]:
# Aceptamos las cookies 
acepto = driver.find_element(By.ID, 'didomi-notice-agree-button')
acepto.click()

In [7]:
# Seleccionamos la barra y escribimos Madrid 
# Por ralguna razon el codigo solo me va cuando esta abierta la ventana del chrome a mitad 
# Hay algo en el codigo de la página que hace que el ID pase de ser 'autocomplete-0-input' a 'autocomplete-1-input'

text = driver.find_element(By.ID, 'autocomplete-0-input')

text.click()
time.sleep(2)
text.send_keys('Madrid')
time.sleep(2)
text.send_keys(Keys.ENTER)

In [8]:
# Sacamos el nombre del restaurante 

titulo = driver.find_element(By.CSS_SELECTOR, 'h3')
print(titulo.text)

Paco Roncero


In [9]:
# Sacamos la ubicacion del restaurante 

ubi = driver.find_element(By.XPATH, '/html/body/main/section[1]/div/div/div[2]/div[14]/div/div[2]/div[2]')
print(ubi.text)

Madrid, España


In [10]:
# Sacamos los euroes y el estilo del restaurante 

ubi = driver.find_element(By.XPATH, '/html/body/main/section[1]/div/div/div[2]/div[1]/div/div[2]/div[3]')
print(ubi.text)

€€€€ · Creativa


In [11]:
# Cogemos todos los links que hay en la página
# Hacemos una lista que tenga todos los URLS disponibles de la página para quedarnos con ellos 

urls = []
for i in range(1,10,1):
    link = f'https://guide.michelin.com/es/es/comunidad-de-madrid/madrid/restaurantes/page/{i}'
    urls.append(link)

In [37]:
# Hacemos el bucle para sacar los 163 datos


lista_nombres = []
lista_ubis = []
lista_estilos = []

for url in urls[:]:
    # Ponemos el servidor donde queremos navegar 

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    time.sleep(2)
    driver.get(url)
    time.sleep(2)

    # Le damos click al boton de cookies 
    acepto = driver.find_element(By.ID, 'didomi-notice-agree-button')
    acepto.click()

    # Empezamos a crear los dataframes
    nombre = driver.find_elements(By.CSS_SELECTOR, 'h3')
    nombres = []
    for titol in nombre:
        nombres.append(titol.text)

    if url != 'https://guide.michelin.com/es/es/comunidad-de-madrid/madrid/restaurantes/page/9':
        for i in range(1,21,1):
            ubi = driver.find_element(By.XPATH, value = f'/html/body/main/section[1]/div/div/div[2]/div{[i]}/div/div[2]/div[2]')
            ubicacion = []
            a = ubi.text
            ubicacion.append(a)
            lista_ubis.append(ubicacion)
    else: 
        for i in range(1,4,1):
            ubi = driver.find_element(By.XPATH, value = f'/html/body/main/section[1]/div/div/div[2]/div{[i]}/div/div[2]/div[2]')
            ubicacion = []
            a = ubi.text
            ubicacion.append(a)
            lista_ubis.append(ubicacion)


    if url != 'https://guide.michelin.com/es/es/comunidad-de-madrid/madrid/restaurantes/page/9':
        for i in range(1,21,1):
            estilo = driver.find_element(By.XPATH, value = f'/html/body/main/section[1]/div/div/div[2]/div{[i]}/div/div[2]/div[3]')
            style = []
            b = estilo.text
            style.append(b)
            lista_estilos.append(style)
    else: 
        for i in range(1,4,1):
            estilo = driver.find_element(By.XPATH, value = f'/html/body/main/section[1]/div/div/div[2]/div{[i]}/div/div[2]/div[3]')
            style = []
            b = estilo.text
            style.append(b)
            lista_estilos.append(style)

    lista_nombres.append(nombres)
    print(url)
    time.sleep(1)


https://guide.michelin.com/es/es/comunidad-de-madrid/madrid/restaurantes/page/1
https://guide.michelin.com/es/es/comunidad-de-madrid/madrid/restaurantes/page/2
https://guide.michelin.com/es/es/comunidad-de-madrid/madrid/restaurantes/page/3
https://guide.michelin.com/es/es/comunidad-de-madrid/madrid/restaurantes/page/4
https://guide.michelin.com/es/es/comunidad-de-madrid/madrid/restaurantes/page/5
https://guide.michelin.com/es/es/comunidad-de-madrid/madrid/restaurantes/page/6
https://guide.michelin.com/es/es/comunidad-de-madrid/madrid/restaurantes/page/7
https://guide.michelin.com/es/es/comunidad-de-madrid/madrid/restaurantes/page/8
https://guide.michelin.com/es/es/comunidad-de-madrid/madrid/restaurantes/page/9


In [45]:
# Creamos un dataframe a mano de los restaurantes que tienen estrella 

estrellas = [2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,
             0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,2,0,0,0,0,1,0,0,0,0,0,0,
             0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,
             1,0,0,1,0,0,0,0,0,1,0,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,
             0,0,0]

In [39]:
# Hacemos que las listas sean de una sola dimension

lista_names = []
lista_location = []
lista_style = []

for i in lista_nombres:
    for j in i:
        lista_names.append(j)

for i in lista_ubis:
    for j in i:
        lista_location.append(j)

for i in lista_estilos:
    for j in i:
        lista_style.append(j)


lista_names = [r for r in lista_names if r !='']

In [46]:
# Creamos el dataframe 

data = {'Nombres': lista_names,
        'Restaurante': lista_location,
        'Estilo/Precio': lista_style, 
        'Estrellas Michelin': estrellas}

Restaurantes = pd.DataFrame(data)
Restaurantes.tail(10)

,Nombres,Restaurante,Estilo/Precio,Estrellas Michelin
153,El Viejo Fogón,"Majadahonda, España",€€ · Tradicional,0
154,El Quinto Sabor,"Villaviciosa de Odón, España",€€ · Creativa,0
155,La Terraza de Alba,"Tres Cantos, España",€€ · Tradicional,0
156,La Sartén,"Tres Cantos, España",€€ · Fusión,0
157,Chirón,"Valdemoro, España",€€€ · Moderna,1
158,El Bistró,"Griñón, España",€€ · Tradicional,0
159,El Trasgu,"Torrelodones, España",€€€ · Tradicional,0
160,Eximio by Fernando Martín,"Alcalá de Henares, España",€€ · Actual,0
161,El Vagón de Beni,"Hoyo de Manzanares, España",€€ · Tradicional,0
162,Ki-Jote,"Alcalá de Henares, España",€€ · Japonesa,0


In [47]:
# Hacemos un split de la columna Estilo/Precio para quedarnos con dos variables del estilo y del precio 
Restaurantes[['Precio', 'Estilo']] = Restaurantes['Estilo/Precio'].str.split('·', n=1, expand=True)
Restaurantes = Restaurantes.drop(columns= 'Estilo/Precio')

In [48]:
Restaurantes

,Nombres,Restaurante,Estrellas Michelin,Precio,Estilo
0,Paco Roncero,"Madrid, España",2,€€€€,Creativa
1,CEBO,"Madrid, España",0,€€€€,Creativa
2,Umiko,"Madrid, España",0,€€€,Japonesa
3,Casa Mortero,"Madrid, España",0,€€,Creativa
4,Baoli,"Madrid, España",0,€€€,China contemporánea
...,...,...,...,...,...
158,El Bistró,"Griñón, España",0,€€,Tradicional
159,El Trasgu,"Torrelodones, España",0,€€€,Tradicional
160,Eximio by Fernando Martín,"Alcalá de Henares, España",0,€€,Actual
161,El Vagón de Beni,"Hoyo de Manzanares, España",0,€€,Tradicional


In [49]:
Restaurantes.groupby('Estilo').nunique().info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries,  A la Parrilla to  Yakitori
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Nombres             36 non-null     int64
 1   Restaurante         36 non-null     int64
 2   Estrellas Michelin  36 non-null     int64
 3   Precio              36 non-null     int64
dtypes: int64(4)
memory usage: 1.4+ KB


## Creación del Recomendador 

- Primero haremos un One Hot Encoder para sacar features de los restaurantes 
- Luego utilizaremos un KNN para buscar los restaurantes que más se parecen entre ellos

### One Hot Encoder

- Usaremos un OneHotEncoder para crear más features


In [50]:
# Hacemos un df que sea identico al originl para crear los nuevos features 

res = Restaurantes
ohe = OneHotEncoder()

In [51]:
# Hacemos el one-hot Encoder para nuestro dataset 

featura_array = ohe.fit_transform(res[['Estilo', 'Precio']]).toarray()

In [52]:
#  Guardamos los labels en una variable 
feature_labels = ohe.categories_

In [53]:
np.array(feature_labels, dtype= 'object').ravel()

array([array([' A la Parrilla', ' Actual', ' Andaluza', ' Argentina',
              ' Asturiana', ' Carnes a la parrilla', ' Castellana', ' China',
              ' China contemporánea', ' Clásica', ' Cocina de mercado',
              ' Cocina de temporada', ' Cocina mediterránea', ' Colombiana',
              ' Creativa', ' Española', ' Francesa', ' Francesa moderna',
              ' Fusión', ' Gallega', ' Hindú', ' Innovadora', ' Internacional',
              ' Italiana', ' Japonesa', ' Japonesa contemporánea', ' Mexicana',
              ' Moderna', ' Peruana', ' Pescados y mariscos', ' Regional',
              ' Steakhouse japonesa', ' Tradicional', ' Vasca', ' Vegetariana',
              ' Yakitori'], dtype=object)                                      ,
       array(['€ ', '€€ ', '€€€ ', '€€€€ '], dtype=object)], dtype=object)

In [54]:
feature_label = np.array(feature_labels, dtype='object').ravel()

In [55]:
labels = []
for i in feature_label:
    for j in i:
        labels.append(j)

In [56]:
print(labels)

[' A la Parrilla', ' Actual', ' Andaluza', ' Argentina', ' Asturiana', ' Carnes a la parrilla', ' Castellana', ' China', ' China contemporánea', ' Clásica', ' Cocina de mercado', ' Cocina de temporada', ' Cocina mediterránea', ' Colombiana', ' Creativa', ' Española', ' Francesa', ' Francesa moderna', ' Fusión', ' Gallega', ' Hindú', ' Innovadora', ' Internacional', ' Italiana', ' Japonesa', ' Japonesa contemporánea', ' Mexicana', ' Moderna', ' Peruana', ' Pescados y mariscos', ' Regional', ' Steakhouse japonesa', ' Tradicional', ' Vasca', ' Vegetariana', ' Yakitori', '€ ', '€€ ', '€€€ ', '€€€€ ']


In [57]:
features = pd.DataFrame(featura_array, columns= labels)
features

,A la Parrilla,Actual,Andaluza,Argentina,Asturiana,Carnes a la parrilla,Castellana,China,China contemporánea,Clásica,...,Regional,Steakhouse japonesa,Tradicional,Vasca,Vegetariana,Yakitori,€,€€,€€€,€€€€
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
160,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [58]:
res = pd.concat([res, features ], axis = 1)
res


,Nombres,Restaurante,Estrellas Michelin,Precio,Estilo,A la Parrilla,Actual,Andaluza,Argentina,Asturiana,...,Regional,Steakhouse japonesa,Tradicional,Vasca,Vegetariana,Yakitori,€,€€,€€€,€€€€
0,Paco Roncero,"Madrid, España",2,€€€€,Creativa,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,CEBO,"Madrid, España",0,€€€€,Creativa,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Umiko,"Madrid, España",0,€€€,Japonesa,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,Casa Mortero,"Madrid, España",0,€€,Creativa,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,Baoli,"Madrid, España",0,€€€,China contemporánea,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,El Bistró,"Griñón, España",0,€€,Tradicional,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
159,El Trasgu,"Torrelodones, España",0,€€€,Tradicional,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
160,Eximio by Fernando Martín,"Alcalá de Henares, España",0,€€,Actual,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
161,El Vagón de Beni,"Hoyo de Manzanares, España",0,€€,Tradicional,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [59]:
# Pasamos el dataframe transfromado a csv para el flask
Restaurantes.to_csv(r'C:\Users\Gabriel Castillo\Desktop\Cosas Gabriel\ML Restaurantes\res.csv')
features.to_csv(r'C:\Users\Gabriel Castillo\Desktop\Cosas Gabriel\ML Restaurantes\features.csv', index=False)

In [60]:
# Aquí tenemos dos opciones, haremos un nearest neighbor para ver como sale y luego probaremos con un KNN para ver que sale mejor 

n_neighbors = 5

nneighbors = NearestNeighbors(n_neighbors= n_neighbors, metric = 'cosine').fit(features)

In [61]:
Restaurantes.iloc[68]

Nombres                      Arzábal
Restaurante           Madrid, España
Estrellas Michelin                 2
Precio                           €€ 
Estilo                   Tradicional
Name: 68, dtype: object

In [62]:
features.iloc[[12]]

,A la Parrilla,Actual,Andaluza,Argentina,Asturiana,Carnes a la parrilla,Castellana,China,China contemporánea,Clásica,...,Regional,Steakhouse japonesa,Tradicional,Vasca,Vegetariana,Yakitori,€,€€,€€€,€€€€
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [63]:
dif, ind = nneighbors.kneighbors(features.iloc[[2]])

print('Liked Restaurant')
print ( '='*60)
print(Restaurantes.loc[ind[0][0], :])
print("")
print('Recommended Restaurants')
print ( '='*60)
print(Restaurantes.loc[ind[0][1:4], :])

Liked Restaurant
Nombres                               Kabutokaji
Restaurante           Pozuelo de Alarcón, España
Estrellas Michelin                             0
Precio                                      €€€ 
Estilo                                  Japonesa
Name: 145, dtype: object
Recommended Restaurants
              Nombres         Restaurante  Estrellas Michelin Precio  \
116  Ikigai Velázquez      Madrid, España                   0   €€€    
32   Ikigai Flor Baja      Madrid, España                   0   €€€    
150      99 sushi bar  Alcobendas, España                   0   €€€    

        Estilo  
116   Japonesa  
32    Japonesa  
150   Japonesa  


In [64]:
# Vemos cuales son los 5 restuarantes que más se le parecen 

ind

array([[145, 116,  32, 150,  97]], dtype=int64)

In [65]:
# Guardamos el modelo 

filename = 'Modelo_Restaurantes'
pickle.dump(nneighbors, open(filename, 'wb'))